# Hospital Readmission Exploratory Analysis

In [4]:
import polars as pl
import altair as alt
import kagglehub
import os


In [23]:
# Download dataset
test_path = kagglehub.dataset_download("vanpatangan/readmission-dataset", path="test_df.csv")
train_path = kagglehub.dataset_download("vanpatangan/readmission-dataset", path="train_df.csv")

print(test_path)
print(train_path)


C:\Users\deric\.cache\kagglehub\datasets\vanpatangan\readmission-dataset\versions\1\test_df.csv
C:\Users\deric\.cache\kagglehub\datasets\vanpatangan\readmission-dataset\versions\1\train_df.csv


In [21]:
test_df = pl.read_csv(test_path)
print(test_df.head())

shape: (5, 7)
┌─────┬────────┬────────────────┬────────────────┬────────────────┬────────────────┬───────────────┐
│ age ┆ gender ┆ primary_diagno ┆ num_procedures ┆ days_in_hospit ┆ comorbidity_sc ┆ discharge_to  │
│ --- ┆ ---    ┆ sis            ┆ ---            ┆ al             ┆ ore            ┆ ---           │
│ i64 ┆ str    ┆ ---            ┆ i64            ┆ ---            ┆ ---            ┆ str           │
│     ┆        ┆ str            ┆                ┆ i64            ┆ i64            ┆               │
╞═════╪════════╪════════════════╪════════════════╪════════════════╪════════════════╪═══════════════╡
│ 52  ┆ Male   ┆ Heart Disease  ┆ 3              ┆ 9              ┆ 3              ┆ Home          │
│ 47  ┆ Female ┆ Diabetes       ┆ 2              ┆ 4              ┆ 0              ┆ Skilled       │
│     ┆        ┆                ┆                ┆                ┆                ┆ Nursing       │
│     ┆        ┆                ┆                ┆                ┆          

In [22]:
train_df = pl.read_csv(train_path)
print(train_df.head())

shape: (5, 8)
┌─────┬────────┬──────────────┬─────────────┬─────────────┬─────────────┬─────────────┬────────────┐
│ age ┆ gender ┆ primary_diag ┆ num_procedu ┆ days_in_hos ┆ comorbidity ┆ discharge_t ┆ readmitted │
│ --- ┆ ---    ┆ nosis        ┆ res         ┆ pital       ┆ _score      ┆ o           ┆ ---        │
│ i64 ┆ str    ┆ ---          ┆ ---         ┆ ---         ┆ ---         ┆ ---         ┆ i64        │
│     ┆        ┆ str          ┆ i64         ┆ i64         ┆ i64         ┆ str         ┆            │
╞═════╪════════╪══════════════╪═════════════╪═════════════╪═════════════╪═════════════╪════════════╡
│ 69  ┆ Male   ┆ Heart        ┆ 1           ┆ 2           ┆ 1           ┆ Home Health ┆ 0          │
│     ┆        ┆ Disease      ┆             ┆             ┆             ┆ Care        ┆            │
│ 32  ┆ Female ┆ COPD         ┆ 2           ┆ 13          ┆ 2           ┆ Rehabilitat ┆ 0          │
│     ┆        ┆              ┆             ┆             ┆             ┆ ion

# Basic Descriptions of Training and Testing Data

In [25]:
train_df.describe()

statistic,age,gender,primary_diagnosis,num_procedures,days_in_hospital,comorbidity_score,discharge_to,readmitted
str,f64,str,str,f64,f64,f64,str,f64
"""count""",5000.0,"""5000""","""5000""",5000.0,5000.0,5000.0,"""5000""",5000.0
"""null_count""",0.0,"""0""","""0""",0.0,0.0,0.0,"""0""",0.0
"""mean""",53.299,null,null,4.461,7.3966,2.0686,null,0.188
"""std""",20.646851,null,null,2.88606,4.025587,1.422357,null,0.390751
"""min""",18.0,"""Female""","""COPD""",0.0,1.0,0.0,"""Home""",0.0
"""25%""",36.0,null,null,2.0,4.0,1.0,null,0.0
"""50%""",53.0,null,null,4.0,7.0,2.0,null,0.0
"""75%""",71.0,null,null,7.0,11.0,3.0,null,0.0
"""max""",89.0,"""Male""","""Kidney Disease""",9.0,14.0,4.0,"""Skilled Nursing Facility""",1.0


In [26]:
test_df.describe()

statistic,age,gender,primary_diagnosis,num_procedures,days_in_hospital,comorbidity_score,discharge_to
str,f64,str,str,f64,f64,f64,str
"""count""",2000.0,"""2000""","""2000""",2000.0,2000.0,2000.0,"""2000"""
"""null_count""",0.0,"""0""","""0""",0.0,0.0,0.0,"""0"""
"""mean""",54.318,null,null,4.563,7.4195,2.053,null
"""std""",20.872631,null,null,2.846767,3.99944,1.40932,null
"""min""",18.0,"""Female""","""COPD""",0.0,1.0,0.0,"""Home"""
"""25%""",36.0,null,null,2.0,4.0,1.0,null
"""50%""",55.0,null,null,5.0,7.0,2.0,null
"""75%""",72.0,null,null,7.0,11.0,3.0,null
"""max""",89.0,"""Male""","""Kidney Disease""",9.0,14.0,4.0,"""Skilled Nursing Facility"""
